In [21]:
import pandas as pd
import json
from pathlib import Path
from json_flatten import flatten
import numpy as np
import torch
import os

In [22]:
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

In [23]:
# Get the current working directory
current_directory = os.getcwd()

In [18]:
songs_meta_data =  pd.read_json(fr'{current_directory}\Melon - playlist dataset\song_meta.json')
playlists = pd.read_json(fr'{current_directory}\Melon - playlist dataset\train.json')

### Set the artist you want to get its related artists (Seed-artist)

In [33]:
artist = 'jamiroquai'

#Add bool col that mask if the chosen artist is related to this song ( in each row ) 
songs_meta_data['is_artist'] = songs_meta_data.apply(lambda row:any(artist in string.lower() for string in row['artist_name_basket']),axis = 1)

#### Check if the artist exists in the data


In [32]:
if songs_meta_data['is_artist'].sum()>0:
    print('Artist exists in data')
else:
    print('Artist do not exists in data')

Artist do not exists in data


#### Get list of indexes of the seed artist songs 

In [33]:
artist_songs_index = songs_meta_data[songs_meta_data['is_artist']].index.to_list()

#### Build dictionary with all the artists in the dataset

In [36]:
artists = {}

In [37]:
for plst in playlists.index:
    curr_plst = songs_meta_data.loc[playlists.iloc[plst,:]['songs']]
    arst_bckt = curr_plst['artist_name_basket']
    for i in arst_bckt:
        for j in i:
            if j in artists.keys():
                artists[j]+=1
            else:
                artists[j] = 1

#### Get playlists index which contains the seed artist

In [38]:
playlists_contains_artist = []
for i in range(playlists.shape[0]):
    if any(x in artist_songs_index for x in playlists.iloc[i,3]):
        playlists_contains_artist.append(i)

#### Get the artists that share playlists with the seed artist

In [39]:
related_artists = {}

In [40]:
for plst in playlists_contains_artist:
    curr_plst = songs_meta_data.loc[playlists.iloc[plst,:]['songs']]
    arst_bckt = curr_plst['artist_name_basket']
    for i in arst_bckt:
        for j in i:
            if j in related_artists.keys():
                related_artists[j]+=1
            else:
                related_artists[j] = 1

In [41]:
related_artists_df = pd.DataFrame.from_dict(related_artists,orient='index').reset_index()
related_artists_df.rename(columns={"index": "Artist", 0: "Joint appearance"},inplace = True)
related_artists_df['Artist'] = related_artists_df['Artist'].apply(lambda x:x.lower())

In [36]:
related_artists_df = related_artists_df.sort_values(by=['Joint appearance'],ascending = False)

In [43]:
related_artists_df

,Artist,Joint appearance
12,jamiroquai,1832
145,maroon 5,653
245,coldplay,468
10,jason mraz,452
192,bruno mars,430
241,jeff bernat,384
235,john legend,352
131,daft punk,323
206,ed sheeran,312
316,norah jones,288


#### Export the related artist file into csv

In [44]:
related_artists_df.to_csv(fr'{current_directory}/related_artists_from_melon/{artist}_related_artists_df.csv')

---------------------------------

# Extra analysis - not necessary for the experiment

#### The most popular artists with seed artist in playlists

In [23]:
most_related_artists = list(related_artists.keys())[:100]

In [24]:
most_related_artists

['Kurt Masur',
 'Maxim Vengerov',
 'Ton Koopman',
 'Bolshoi Theater Symphony Orchestra',
 'Ludwig Van Beethoven',
 'Wolfgang Amadeus Mozart',
 'George Mann',
 'Wolfgang Amadeus Mozart ',
 'Vladimir Horowitz',
 'Various Artists',
 'Antonio Vivaldi',
 'Peter Nagy',
 'ThirdWave',
 'Claude Debussy',
 'Johann Sebastian Bach',
 'Acoustic Cafe',
 'Erik Satie',
 'Mikhail Rudy',
 'Mariss Jansons',
 'St. Petersburg Philharmonic Orchestra',
 'Classical Music',
 'Franz Peter Schubert',
 '클래식피아노 베스트',
 'Jascha Heifetz',
 '조성진',
 'Yo-Yo Ma',
 'Emanuel Ax',
 'Itzhak Perlman',
 'Emerson String Quartet',
 'Leon Fleisher',
 'David Shifrin',
 'Amadeus Quartett',
 'Cecil Aronowitz',
 'William Pleeth',
 'Christian Ferras',
 'Pierre Barbizet',
 'Sol Gabetta',
 'H&#233;l&#232;ne Grimaud',
 'Quartetto Italiano',
 'Melos Quartet',
 'Jack Brymer',
 'The Allegri String Quartet',
 'Janacek Quartet',
 'Lang Lang',
 'Vadim Repin',
 'Mischa Maisky',
 'Mstislav Rostropovich',
 'Sviatoslav Richter',
 'David Oistrakh',

In [21]:
related_artists = dict(sorted(related_artists.items(), key=lambda item: item[1],reverse = True))

#### Get the playlists that don't share playlist with the seed artist

In [ ]:
not_related_artists = list(set(artists) - set(related_artists))

In [ ]:
not_related_artists

#### Get songs of specific playlist

In [ ]:
pd.set_option('display.max_rows', 200)
songs_meta_data.loc[playlists.iloc[108109,:]['songs']]

#### Get meta data of  specific playlist

In [ ]:
pd.DataFrame(playlists.iloc[1061,:])